<a href="https://colab.research.google.com/github/prachi428/MIDAS_Task2_2021/blob/main/Task2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/MCA\ Project/MIDAS

/content/drive/MyDrive/MCA Project/MIDAS


In [2]:
import torch, glob, cv2, os, tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split
import random
from skimage import transform
import numpy as np
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# files
# %pwd
image = cv2.imread("/content/drive/MyDrive/MCA Project/MIDAS/train/Sample001/img001-001.png")
image2 = cv2.imread("/content/drive/MyDrive/MCA Project/MIDAS/train/Sample001/img001-055.png")
image3 = cv2.imread("/content/drive/MyDrive/MCA Project/MIDAS/train/Sample061/img061-001.png")
test_set = []
train_set = []
for i in range(int(0.4*62)):
  test_class = random.randint(1,62)
  while(test_class in test_set):
    test_class = random.randint(1, 62)
  test_set.append(test_class)

In [4]:
len(test_set)
for i in range(1, 63):
  if(i not in test_set):
    train_set.append(i)


In [5]:
class GetData(Dataset):

  def __init__(self, indices, dir="train/", transform=None):
    
    self.transform = transform
    self.files = {}
    self.len = 0
    self.ind = indices
    for n in self.ind:
      # print(n)
      self.files[n] = []
      n = str(n)
      if(len(n) < 3):
        for i in range(3 - len(n)):
          n = '0' + n
      for image in glob.glob(dir + "Sample" + n + '/*'):
        self.files[int(n)].append(image) 
        self.len += 1  


  
  def __len__(self):
    return self.len

  
  def __getitem__(self, idx):
    idx_dict = int(idx/40)
    idx_ls = idx - idx_dict*40
    # print(idx_dict)
    # print(idx_ls, len(self.files[idx_dict]))
    img_name = self.files[self.ind[idx_dict]][idx_ls]
    image = cv2.imread(img_name)


    image_pos = random.randint(0, 39)
    if(image_pos == idx_ls):
      image_pos -= 1
    if image_pos == -1:
      image_pos = image_pos + 1
    image_pos = cv2.imread(self.files[self.ind[idx_dict]][idx_ls])


    idx_neg_dict = random.randint(0, len(self.ind)-1)
    idx_neg = random.randint(0, 39)

    if(idx_neg_dict == idx_dict):
      idx_neg_dict -= 1
    if idx_neg_dict == -1:
      idx_neg_dict = idx_neg_dict+2


    image_neg = cv2.imread(self.files[self.ind[idx_neg_dict]][idx_neg])


    pair = {"main": image, "pos": image_pos, "neg": image_neg}
    if self.transform:
      pair = self.transform(pair)
    return pair

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        img1, img2, img3 = sample["main"], sample["pos"], sample["neg"]
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        # print(img1.shape)

        return {"main": torch.from_numpy(img1).type(torch.FloatTensor),
        "pos": torch.from_numpy(img2).type(torch.FloatTensor),
        "neg": torch.from_numpy(img3).type(torch.FloatTensor)}


class Resize(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    # def __init__(self):
    #     assert isinstance(output_size, (int, tuple))
        # self.output_size = output_size

    def __call__(self, sample):
        img1, img2, img3 = sample["main"], sample["pos"], sample["neg"]
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        img1 = transform.resize(img1[:, :, 0], (105, 105))
        img2 = transform.resize(img2[:, :, 0], (105, 105))
        img3 = transform.resize(img3[:, :, 0], (105, 105))
 
        return {"main": img1,
        "pos": img2,
        "neg": img3}

In [6]:
transformed_dataset = GetData(train_set, transform=transforms.Compose([Resize(), ToTensor()]))


In [7]:
len_train = int(0.8*len(transformed_dataset))
train, val = random_split(transformed_dataset, [len_train, int(len(transformed_dataset) - len_train)])
len(transformed_dataset), len(train), len(val)


(1520, 1216, 304)

In [8]:
train_loader = DataLoader(train, batch_size=40, shuffle=True)
val_loader = DataLoader(val, batch_size=40, shuffle=True)

In [9]:
# for t in train_loader:
#   print(t["main"].shape)
#   break

In [10]:
class SiameseNetwork(nn.Module):
  def __init__(self):
    super(SiameseNetwork, self).__init__()

    self.conv_model = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=96, kernel_size=10),
      nn.MaxPool2d(2),
      nn.ReLU(inplace=True),

      
      nn.Conv2d(in_channels=96, out_channels=128, kernel_size=7),
      nn.MaxPool2d(2),
      nn.ReLU(inplace=True),

      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=4),
      nn.MaxPool2d(2),
      nn.ReLU(inplace=True),
      
      nn.Conv2d(in_channels=128, out_channels=256, kernel_size=4),
      # nn.MaxPool2d(2),
      nn.ReLU(inplace=True))
    # for m in self.modules():
    #         if isinstance(m, nn.Conv2d):
    #             nn.init.kaiming_normal(m.weight, mode='fan_in')
    
    self.fc1 = nn.Linear(9216, 4096)
    self.fc2 = nn.Linear(4096, 1)
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
          nn.init.normal_(m.weight, 0, 1e-2)
          nn.init.normal_(m.bias, 0.5, 1e-2)
      elif isinstance(m, nn.Linear):
          nn.init.normal_(m.weight, 0, 2e-1)
          # nn.init.normal(m.weight, 0, 1e-2)

  def sub_forward(self, x):
    x = self.conv_model(x)

    x = x.view(x.size(0), -1)

    x = torch.sigmoid(self.fc1(x))
    return x

  def forward(self, x1, x2):
    x1 = self.sub_forward(x1)
    x2 = self.sub_forward(x2)
    diff = torch.abs(x1 - x2)

    scores = self.fc2(diff)

    return scores

In [18]:
best_valid_acc = 0.
epochs = 200
start_epoch = 0
lr_patience = 1
train_patience = 20
counter = 0
num_model = '1'
ckpt_dir = './ckpt/'
logs_dir = './logs/'
plot_dir = './plots/'
num_train = len(train)
num_valid = len(val)
ckpt_dir = os.path.join(ckpt_dir, num_model)
logs_dir = os.path.join(logs_dir, num_model)
os.makedirs(ckpt_dir)
os.makedirs(logs_dir)
layer_hyperparams = {
            'layer_init_lrs': [],
            'layer_end_momentums': [],
            'layer_l2_regs': []
        }
print("[*] Sampling layer hyperparameters.")

layer_hyperparams = {
    'layer_init_lrs': [],
    'layer_end_momentums': [],
    'layer_l2_regs': []
}
model = SiameseNetwork().to(device)
num_layers = len(list(model.children()))
init_momentum = 0.5
for i in range(6):
    # sample
    lr = random.uniform(1e-4, 1e-1)
    mom = random.uniform(0, 1)
    reg = random.uniform(0, 0.1)

    # store
    layer_hyperparams['layer_init_lrs'].append(lr)
    layer_hyperparams['layer_end_momentums'].append(mom)
    layer_hyperparams['layer_l2_regs'].append(reg)
        
# grab layer-wise hyperparams
init_lrs = layer_hyperparams['layer_init_lrs']
init_momentums = [init_momentum]*num_layers
end_momentums = layer_hyperparams['layer_end_momentums']
l2_regs = layer_hyperparams['layer_l2_regs']

if epochs == 1:
    f = lambda max, min: min
else:
    f = lambda max, min: (max - min) / (epochs-1)
momentum_temper_rates = [
    f(x, y) for x, y in zip(end_momentums, init_momentums)
]

# lrs = init_lrs
# momentums = init_momentums
optimizer = torch.optim.Adam(model.parameters(), lr=3e-2, weight_decay=6e-5)

[*] Sampling layer hyperparameters.


In [22]:
import shutil
def save_checkpoint(state, is_best):
  filename = 'model_ckpt.tar'
  ckpt_path = os.path.join(ckpt_dir, filename)
  torch.save(state, ckpt_path)

  if is_best:
    filename = 'best_model_ckpt.tar'
    shutil.copyfile(ckpt_path, os.path.join(ckpt_dir, filename))

def load_checkpoint(best=False, ckpt_dir = './ckpt/'):
  print("[*] Loading model from {}".format(ckpt_dir))

  filename = 'model_ckpt.tar'
  if best:
      filename = 'best_model_ckpt.tar'
  ckpt_path = os.path.join(ckpt_dir, filename)
  ckpt = torch.load(ckpt_path)

  # load variables from checkpoint
  start_epoch = ckpt['epoch']
  best_valid_acc = ckpt['best_valid_acc']
  model.load_state_dict(ckpt['model_state'])
  optimizer.load_state_dict(ckpt['optim_state'])

  if best:
      print(
          "[*] Loaded {} checkpoint @ epoch {} "
          "with best valid acc of {:.3f}".format(
              filename, ckpt['epoch'], ckpt['best_valid_acc'])
      )
  else:
      print(
          "[*] Loaded {} checkpoint @ epoch {}".format(
              filename, ckpt['epoch'])
      )

In [20]:
# import tqdm
        
        

In [21]:
# if resume:
#     self.load_checkpoint(best=False)

# switch to train mode

# create train and validation log files
optim_file = open(os.path.join(logs_dir, 'optim.csv'), 'w')
train_file = open(os.path.join(logs_dir, 'train.csv'), 'w')
valid_file = open(os.path.join(logs_dir, 'valid.csv'), 'w')

print("\n[*] Train on {} sample pairs, validate on {} trials".format(
    num_train, num_valid)
)

for epoch in range(0, epochs):
    
    print('\nEpoch: {}/{}'.format(epoch+1, epochs))
    model.train()

    with tqdm.tqdm(total=num_train) as pbar:
      for i, sample in enumerate(train_loader):
        # print(len(sample))
        x1, x2, x3 = sample["main"], sample["pos"], sample["neg"]
        x1, x2, x3 = x1.to(device), x2.to(device), x3.to(device)
        x1, x2, x3 = x1.unsqueeze(1), x2.unsqueeze(1), x3.unsqueeze(1)
        # print(x1.shape)
        # x1, x2, x3 = Variable(x1), Variable(x2), Variable(x3)

        # split input pairs along the batch dimension
        batch_size = x1.shape[0]

        out1 = model(x1, x2)
        out2 = model(x1, x3)
        loss1 = F.binary_cross_entropy_with_logits(out1, torch.ones(out1.shape).to(device))
        loss2 = F.binary_cross_entropy_with_logits(out2, torch.zeros(out2.shape).to(device))

        # compute gradients and update
        optimizer.zero_grad()
        loss = loss1 + loss2
        loss.backward()
        # loss2.backward()
        optimizer.step()
        print(loss1.item(), loss2.item())
        # store batch statistics

        pbar.set_description(
            (
                "loss: {:.6f}".format(
                    loss.item(),
                )
            )
        )
        pbar.update(batch_size)

        # log loss
        iter = (epoch * len(train_loader)) + i
        train_file.write('{},{}\n'.format(
            iter, loss.item())
        )

    

    model.eval()
    correct = 0
    for i, sample in enumerate(val_loader):
        x1, x2, x3 = sample["main"], sample["pos"], sample["neg"]
        x1, x2, x3 = x1.to(device), x2.to(device), x3.to(device)
        x1, x2, x3 = x1.unsqueeze(1), x2.unsqueeze(1), x3.unsqueeze(1)

        # x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

        batch_size = x1.shape[0]

        # compute log probabilities
        out1 = model(x1, x2)
        out2 = model(x1, x3)
        log_probas1 = (torch.sigmoid(out1)>0.5)
        log_probas2 = (torch.sigmoid(out2) < 0.5)
        
        for t in log_probas1:
          if t.item():
            correct+=1
        for t in log_probas2:
          if t.item():
            correct+=1

        # get index of max log prob
        # print(log_probas)
        # pred = log_probas.data.max(0)[1][0]

        # if pred == 0:
            # correct += 1

    # compute acc and log
    valid_acc = (100. * correct) / (2*num_valid)
    iter = epoch
    valid_file.write('{},{}\n'.format(
        iter, valid_acc)
    )




    # check for improvement
    is_best = valid_acc > best_valid_acc
    msg = "val acc: {:.6f}"
    if is_best:
        msg += " [*]"
        counter = 0
    print(msg.format(valid_acc))

    # checkpoint the model
    if not is_best:
        counter += 1
    if counter > train_patience:
        print("[!] No improvement in a while, stopping training.")
        break
    best_valid_acc = max(valid_acc, best_valid_acc)
    save_checkpoint(
        {
            'epoch': epoch + 1,
            'model_state': model.state_dict(),
            'optim_state': optimizer.state_dict(),
            'best_valid_acc': best_valid_acc,
        }, is_best
    )
# release resources
optim_file.close()
train_file.close()
valid_file.close()

  0%|          | 0/1216 [00:00<?, ?it/s]


[*] Train on 1216 sample pairs, validate on 304 trials

Epoch: 1/200


loss: 1.304819:   3%|▎         | 40/1216 [00:14<06:51,  2.86it/s]

0.6956269145011902 0.6091917157173157


loss: 1.599971:   7%|▋         | 80/1216 [00:29<06:46,  2.79it/s]

...

0.38402196764945984 0.04527931287884712


loss: 0.379951: 100%|██████████| 1216/1216 [06:54<00:00,  2.93it/s]

0.3731657862663269 0.006785426288843155


val acc: 99.835526 [*]


NameError: ignored

In [ ]:
torch.ones((1,1))
model = SiameseNetwork().to(device)

In [ ]:
for sample in train_loader:
  x1, x2, x3 = sample["main"], sample["pos"], sample["neg"]
  x1, x2, x3 = x1.to(device), x2.to(device), x3.to(device)
  x1, x2, x3 = x1.unsqueeze(1), x2.unsqueeze(1), x3.unsqueeze(1)
  out1 = model(x1, x2)
  out2 = model(x1, x3)
  log_probas1 = (torch.sigmoid(out1))
  log_probas2 = (torch.sigmoid(out2))
  break

In [ ]:
log_probas1

In [30]:
for t in torch.sigmoid(out2):
  print(t.item())

0.013550231233239174
0.27653956413269043
0.011210781522095203
0.004630005918443203
0.0009928708896040916
0.008655696175992489
0.005865841638296843
0.00010199644020758569
0.06163925677537918
0.001496527111157775
0.00010099974315380678
9.675189858171507e-07
0.030920496210455894
0.20706304907798767
0.004100235179066658
0.0011194780236110091
0.012206608429551125
0.04572686180472374
0.004605553112924099
0.0020429904107004404
0.0064968205988407135
0.005293260794132948
1.0012881830334663e-05
4.8767476982902735e-05


In [32]:
transformed_test_dataset = GetData(test_set, transform=transforms.Compose([Resize(), ToTensor()]))
test_loader = DataLoader(transformed_test_dataset, batch_size=40, shuffle=True)


In [33]:
model.eval()
correct = 0
for i, sample in enumerate(test_loader):
    x1, x2, x3 = sample["main"], sample["pos"], sample["neg"]
    x1, x2, x3 = x1.to(device), x2.to(device), x3.to(device)
    x1, x2, x3 = x1.unsqueeze(1), x2.unsqueeze(1), x3.unsqueeze(1)

    # x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    out1 = model(x1, x2)
    out2 = model(x1, x3)
    log_probas1 = (torch.sigmoid(out1)>0.5)
    log_probas2 = (torch.sigmoid(out2) < 0.5)
    
    for t in log_probas1:
      if t.item():
        correct+=1
    for t in log_probas2:
      if t.item():
        correct+=1

    # get index of max log prob
    # print(log_probas)
    # pred = log_probas.data.max(0)[1][0]

    # if pred == 0:
        # correct += 1

# compute acc and log
test_acc = (100. * correct) / (2*num_valid)
iter = epoch
# test_file.write('{},{}\n'.format(
#     iter, valid_acc)
# )

In [38]:
test_acc * (num_valid)/(len(test_loader)*40)

100.0

In [37]:
len(test_loader)

24